In [1]:
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score
from sklearn.preprocessing import Normalizer
import tensorflow as tf
from tqdm import tqdm

In [2]:
print(tf.__version__)

0.12.0


In [3]:
from data import get_data

In [4]:
X_train, y_train, X_test, y_test = get_data(one_hot=True)

In [5]:
mean_of_train = np.mean(X_train)
std_of_train = np.std(X_train)
print(mean_of_train, std_of_train)

115.481949803 65.2925375088


In [6]:
X_train = (X_train - mean_of_train) / std_of_train
X_test = (X_test - mean_of_train) / std_of_train

In [7]:
train_len, width, height, channel = X_train.shape
test_len = X_test.shape[0]
print("""
训练数据数量：{}
高度:{}
宽度：{}
通道数：{}
测试数据数量：{}
""".format(train_len, width, height, channel, test_len))


训练数据数量：20000
高度:128
宽度：128
通道数：3
测试数据数量：5000



In [8]:
# 载入我的自定义库 @qhduan
import sys
import importlib
sys.path.insert(0, '../../')
import tfkit
importlib.reload(tfkit)

<module 'tfkit' from '../../tfkit/__init__.py'>

In [9]:
# 学习率
learning_rate = 0.003
# 迭代次数（批次）
n_epoch = 10
# 批次大小
# 可能需要调小
batch_size = 32
# 输出大小
target_size = y_train.shape[1]
print('target_size is', target_size)

target_size is 2


In [10]:
tf.set_random_seed(0)

In [11]:
# 输入占位符
X = tf.placeholder(tf.float32, [batch_size, width, height, channel])
# 输出占位符
y = tf.placeholder(tf.float32, [batch_size, target_size])

In [12]:
model = X
model = tfkit.conv(model, 64, (5, 5), 'conv_1', strides=(1, 2, 2, 1), activation='relu')
model = tfkit.max_pool(model, 'max_pool_1')
model = tfkit.conv(model, 128, (5, 5), 'conv_2', strides=(1, 2, 2, 1), activation='relu')
model = tfkit.max_pool(model, 'max_pool_2')
model = tfkit.conv(model, 256, (3, 3), 'conv_3', activation='relu')
model = tfkit.max_pool(model, 'max_pool_3')
model = tfkit.flatten(model, 'flatten')
model = tfkit.full_connect(model, 1024, 'fc_1', activation='relu')
model = tfkit.full_connect(model, target_size, 'fc_2', activation='linear')

===> (32, 62, 62, 64) 	 conv_1
===> (32, 31, 31, 64) 	 max_pool_1
===> (32, 14, 14, 128) 	 conv_2
===> (32, 7, 7, 128) 	 max_pool_2
===> (32, 5, 5, 256) 	 conv_3
===> (32, 2, 2, 256) 	 max_pool_3
===> (32, 1024) 	 flatten
===> (32, 1024) 	 fc_1
===> (32, 2) 	 fc_2


In [13]:
train_step, cost = tfkit.train_softmax(
    model, y,
    opt=tf.train.AdadeltaOptimizer(learning_rate=learning_rate)
)

In [14]:
measures = [
    cost,
    tfkit.accuracy(model, y, softmax=True),
]

In [15]:
init = tf.global_variables_initializer()

In [16]:
with tf.Session() as sess:
    sess.run(init)
    total = int((len(X_train) - 1) / batch_size + 1)
    for epoch in range(n_epoch):
        print('epoch: {}'.format(epoch))
        record = []
        for batch_x, batch_y in tqdm(tfkit.batch_flow(X_train, y_train, batch_size), total=total, file=sys.stdout):
            sess.run([train_step], feed_dict={X: batch_x, y: batch_y})
            record.append(sess.run(measures, feed_dict={X: batch_x, y: batch_y}))
        print('train: loss: {:.4f}, acc: {:.4f}'.format(
            np.mean([x[0] for x in record]),
            np.mean([x[1] for x in record])
        ))
        record = []
        for batch_x, batch_y in tfkit.batch_flow(X_test, y_test, batch_size):
            record.append(sess.run(measures, feed_dict={X: batch_x, y: batch_y}))
        print('test: loss: {:.4f}, acc: {:.4f}'.format(
            np.mean([x[0] for x in record]),
            np.mean([x[1] for x in record])
        ))

epoch: 0
100%|██████████| 625/625 [00:16<00:00, 37.03it/s]
train: loss: 1.3212, acc: 0.7492
test: loss: 0.6072, acc: 0.6744
epoch: 1
100%|██████████| 625/625 [00:16<00:00, 37.32it/s]
train: loss: 0.4352, acc: 0.8164
test: loss: 0.5861, acc: 0.6953
epoch: 2
100%|██████████| 625/625 [00:16<00:00, 38.24it/s]
train: loss: 0.3924, acc: 0.8450
test: loss: 0.5722, acc: 0.7108
epoch: 3
100%|██████████| 625/625 [00:16<00:00, 38.35it/s]
train: loss: 0.3524, acc: 0.8663
test: loss: 0.5657, acc: 0.7168
epoch: 4
100%|██████████| 625/625 [00:16<00:00, 38.10it/s]
train: loss: 0.3141, acc: 0.8892
test: loss: 0.5655, acc: 0.7217
epoch: 5
100%|██████████| 625/625 [00:16<00:00, 38.54it/s]
train: loss: 0.2776, acc: 0.9089
test: loss: 0.5673, acc: 0.7221
epoch: 6
100%|██████████| 625/625 [00:16<00:00, 37.71it/s]
train: loss: 0.2420, acc: 0.9263
test: loss: 0.5757, acc: 0.7229
epoch: 7
100%|██████████| 625/625 [00:16<00:00, 37.75it/s]
train: loss: 0.2090, acc: 0.9417
test: loss: 0.5866, acc: 0.7251
epoch: 8